In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import subprocess

In [7]:
def read_gff(path):
    df = pd.read_csv(path, index_col=False, sep='\t', header=None, comment="#")
    df.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
    return df

def sys(command):
    """
    """
    print("-" * 10)
    print("Executing: %s" % command)
    process = subprocess.Popen(command,shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    print("Result: stdout: %s - stderr: %s" % (stdout, stderr))
    print("-" * 10)
    return stdout, stderr



def gff2FA(df_gff, path_sequence, windows, output):
    """Extract fasta files from annotations
    """
    fasta_seq = SeqIO.parse(path_genome, 'fasta')
    buffer_seqs = []
    cont = 0
    for record in fasta_seq:
        print(record.id)
        dff_extract = df_gff[df_gff.seqname == record.id]
        for key,val in dff_extract.iterrows():
            clean_seq = ''.join(str(record.seq).splitlines())
            if int(val.start) - windows < 0:
                start = 0
            else:
                start = int(val.start) - windows
            if int(val.end) + windows > len(clean_seq):
                end = len(clean_seq)
            else:
                end = int(val.end) + windows
            new_seq = clean_seq[start:end]
            att = val.attribute
            id = record.id + '_' + str(start) + '_' + str(end)
            desc = "seq_id:" + str(record.id)
            desc += " feature_start:" + str(val.start)
            desc += " feature_end:" + str(val.end)
            desc += " genome_start:" + str(start)
            desc += " genome_end:" + str(end)
            desc += " feature:" + str(val.feature)
            desc += " attributes:" + val.attribute
            seq = SeqRecord(Seq(new_seq), id=id, description=desc)
            buffer_seqs.append(seq)
            cont += 1
    SeqIO.write(buffer_seqs, output, "fasta")
    return buffer_seqs


In [14]:
functional_annotation = "data/genomes/iwgsc_refseqv1.0_FunctionalAnnotation_v1__HCgenes_v1.0.TAB"
functional_annotation_filtered = "data/genomes/alpha_gliadins.functional.csv"
temp_1 = "data/genomes/t1.csv"
temp_2 = "data/genomes/t2.csv"
annotation = "/home/juan/Desktop/juan/bio/data/IWGSC/42/Triticum_aestivum.IWGSC.42.clean.gff3"
path_genome = "/home/juan/Desktop/juan/bio/data/IWGSC/42/Triticum_aestivum.IWGSC.dna.toplevel.fa"
path_output = "data/gliadings.fa"
path_output_pep = "data/gliadings.pep.fa"
path_pep = "/home/juan/Desktop/juan/bio/data/IWGSC/42/Triticum_aestivum.IWGSC.pep.all.fa"


In [9]:
cmd = "head -n 1 %s > %s" % (functional_annotation, temp_1)
sys(cmd)
cmd = "grep -i 'Alpha-gliadin' %s > %s" % (functional_annotation, temp_2)
sys(cmd)
cmd = "cat %s %s > %s" % (temp_1, temp_2, functional_annotation_filtered)
sys(cmd)


----------
Executing: head -n 1 data/genomes/iwgsc_refseqv1.0_FunctionalAnnotation_v1__HCgenes_v1.0.TAB > data/genomes/t1.csv
Result: stdout: b'' - stderr: b''
----------
----------
Executing: grep -i 'Alpha-gliadin' data/genomes/iwgsc_refseqv1.0_FunctionalAnnotation_v1__HCgenes_v1.0.TAB > data/genomes/t2.csv
Result: stdout: b'' - stderr: b''
----------
----------
Executing: cat data/genomes/t1.csv data/genomes/t2.csv > data/genomes/alpha_gliadins.functional.csv
Result: stdout: b'' - stderr: b''
----------


(b'', b'')

In [10]:

df_func_ann = pd.read_csv(functional_annotation_filtered, sep="\t")
df_func_ann['Gene-ID'] = df_func_ann['Gene-ID'].replace("01G", "02G",regex = True)
print(len(df_func_ann.index))
df_func_ann.head(2)

26


,Gene-ID,is_repr,AHRD-Quality-Code,Blast-Hit-Accession,Human-Readable-Description,Pfam-IDs-(Description),Interpro-IDs-(Description),GO-IDs-(Description)-via-Interpro,Gene-or-TE-TE?-U (via-function),Pfam-IDs,Interpro-IDs,GO-IDs-via-Interpro
0,TraesCS6A02G048900.1,1,*-*,tr|A0A0E3UQY3|A0A0E3UQY3_AEGSE,Alpha-gliadin,PF00234: Protease inhibitor/seed storage/LTP f...,IPR001954: Gliadin/LMW glutenin; IPR016140: Bi...,GO:0045735 MF: nutrient reservoir activity,G,PF00234,IPR001954; IPR016140,GO:0045735
1,TraesCS6A02G049100.1,1,***,tr|A0A0E3Z6W2|A0A0E3Z6W2_9POAL,Alpha-gliadin,PF00234: Protease inhibitor/seed storage/LTP f...,IPR001954: Gliadin/LMW glutenin; IPR016140: Bi...,GO:0045735 MF: nutrient reservoir activity,G,PF00234,IPR001954; IPR016140,GO:0045735


In [11]:
#read and accomodate gff
df_gff = read_gff(annotation)
print(len(df_gff.index))
df_gff = df_gff[df_gff.feature=='mRNA']
print(len(df_gff.index))
df_gff['transcript'] = df_gff['attribute'].str.split('transcript:').str[1]
df_gff['transcript'] = df_gff['transcript'].str.split(';').str[0]
df_gff.head(2)

1957722
133744


,seqname,source,feature,start,end,score,strand,frame,attribute,transcript
34,1A,IWGSC,mRNA,40098,70338,.,-,.,ID=transcript:TraesCS1A02G000100.1;Parent=gene...,TraesCS1A02G000100.1
65,1A,IWGSC,mRNA,70239,89245,.,+,.,ID=transcript:TraesCS1A02G000200.1;Parent=gene...,TraesCS1A02G000200.1


In [12]:
#filter gff gliadins
gliadins_transcripts = df_func_ann['Gene-ID'].tolist()
df_gff = df_gff[df_gff['transcript'].isin(gliadins_transcripts)]
print(len(df_gff.index))
df_gff.head(2)

26


,seqname,source,feature,start,end,score,strand,frame,attribute,transcript
1421508,6A,IWGSC,mRNA,24921651,24922607,.,+,.,ID=transcript:TraesCS6A02G048900.1;Parent=gene...,TraesCS6A02G048900.1
1421531,6A,IWGSC,mRNA,25107550,25108401,.,+,.,ID=transcript:TraesCS6A02G049100.1;Parent=gene...,TraesCS6A02G049100.1


In [16]:
gliadins_transcripts

['TraesCS6A02G048900.1',
 'TraesCS6A02G049100.1',
 'TraesCS6A02G049200.1',
 'TraesCS6A02G049400.1',
 'TraesCS6A02G049500.1',
 'TraesCS6A02G049600.1',
 'TraesCS6A02G049700.1',
 'TraesCS6A02G049800.1',
 'TraesCS6B02G065600.1',
 'TraesCS6B02G065800.1',
 'TraesCS6B02G065900.1',
 'TraesCS6B02G066000.1',
 'TraesCS6B02G066100.1',
 'TraesCS6B02G086500.1',
 'TraesCSU02G108100.1',
 'TraesCSU02G108200.1',
 'TraesCSU02G108300.1',
 'TraesCSU02G108400.1',
 'TraesCSU02G108500.1',
 'TraesCSU02G108600.1',
 'TraesCSU02G153800.1',
 'TraesCSU02G160200.1',
 'TraesCSU02G188800.1',
 'TraesCSU02G220200.1',
 'TraesCSU02G220600.1',
 'TraesCSU02G239000.1']

In [13]:
sequences = gff2FA(df_gff, path_genome, 0, path_output)


1A
1B
1D
2A
2B
2D
3A
3B
3D
4A
4B
4D
5A
5B
5D
6A
6B
6D
7A
7B
7D
Un


In [17]:
#save pepts
fasta_seq = SeqIO.parse(path_pep, 'fasta')
buffer_seqs = []
cont = 0
for record in fasta_seq:
    for gliadins_transcript in gliadins_transcripts:
        if gliadins_transcript in record.id or gliadins_transcript in record.description:
            buffer_seqs.append(record)
            cont += 1
SeqIO.write(buffer_seqs, path_output_pep, "fasta")
print(cont)


19
